In [144]:
import pandas as pd
import numpy as np
import folium
import geopandas as gpd

data = pd.read_csv("data/transactions.csv")

data['Date of Transaction'] = pd.to_datetime(data['Date of Transaction'])
data['Date'] = data['Date of Transaction'].dt.date

data

,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,Money Laundering Risk Score,Shell Companies Involved,Financial Institution,Tax Haven Country,Date
0,TX0000000001,Brazil,3.267530e+06,Offshore Transfer,2013-01-01 00:00:00,Person_1101,Construction,USA,True,Illegal,6,1,Bank_40,Singapore,2013-01-01
1,TX0000000002,China,4.965767e+06,Stocks Transfer,2013-01-01 01:00:00,Person_7484,Luxury Goods,South Africa,False,Illegal,9,0,Bank_461,Bahamas,2013-01-01
2,TX0000000003,UK,9.416750e+04,Stocks Transfer,2013-01-01 02:00:00,Person_3655,Construction,Switzerland,True,Illegal,1,3,Bank_387,Switzerland,2013-01-01
3,TX0000000004,UAE,3.864201e+05,Cash Withdrawal,2013-01-01 03:00:00,Person_3226,Oil & Gas,Russia,False,Illegal,7,2,Bank_353,Panama,2013-01-01
4,TX0000000005,South Africa,6.433784e+05,Cryptocurrency,2013-01-01 04:00:00,Person_7975,Real Estate,USA,True,Illegal,1,9,Bank_57,Luxembourg,2013-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,TX0000009996,Singapore,2.209229e+06,Offshore Transfer,2014-02-21 11:00:00,Person_6896,Real Estate,Brazil,False,Illegal,4,4,Bank_258,Luxembourg,2014-02-21
9996,TX0000009997,UAE,4.800338e+06,Property Purchase,2014-02-21 12:00:00,Person_6348,Luxury Goods,Russia,False,Legal,10,2,Bank_449,Luxembourg,2014-02-21
9997,TX0000009998,UK,4.891953e+05,Stocks Transfer,2014-02-21 13:00:00,Person_4171,Oil & Gas,Russia,False,Illegal,5,0,Bank_156,Cayman Islands,2014-02-21
9998,TX0000009999,Brazil,2.233581e+06,Offshore Transfer,2014-02-21 14:00:00,Person_2799,Real Estate,Russia,True,Illegal,10,5,Bank_12,Singapore,2014-02-21


In [ ]:
#import polygons from geopandas
from shapely.geometry import Point, Polygon, MultiPolygon

#create the polygon for singapore
import osmnx as ox
import geopandas as gpd

gdf = ox.geocode_to_gdf("Singapore")   # País
gdf = gdf.to_crs("EPSG:4326")          # lat/lon
poly = gdf.loc[0, "geometry"]


singapore = {'admin': 'Singapore', 'iso_a3': 'SGP', 'geometry': poly}

In [146]:
url = "https://cdn.jsdelivr.net/gh/gavinr/world-countries-centroids@v1/dist/countries.geojson"

# Cargar el GeoJSON directamente
world = gpd.read_file('data/custom.geo.json')

# Filtrar los países deseados
countries = ['United States of America', 'South Africa', 'Switzerland', 'Russia', 
             'Brazil', 'United Kingdom', 'India', 'China', 'Singapore', 
             'United Arab Emirates']

gdf_countries = world[world['admin'].isin(countries)]
gdf_countries = gdf_countries.drop(columns=['featurecla', 'scalerank', 'labelrank', 'sovereignt', 'sov_a3', 'adm0_dif', 'level'])
gdf_countries = gdf_countries[['admin', 'geometry', 'iso_a3']]

mapping_countries = {
    'Brazil': 'Brazil',
    'China': 'China',
    'India': 'India',
    'Russia': 'Russia',
    'Singapore': 'Singapore',
    'South Africa': 'South Africa',
    'Switzerland': 'Switzerland',
    'United Arab Emirates': 'UAE',
    'United Kingdom': 'UK',
    'United States of America': 'USA'}

gdf_countries['admin'] = gdf_countries['admin'].map(mapping_countries)
gdf_countries.loc[len(gdf_countries)] = singapore

gdf_countries['rep_point'] = gdf_countries['geometry'].representative_point()
gdf_countries.set_crs(epsg=4326, inplace=True)


,admin,geometry,iso_a3,rep_point
8,USA,"MULTIPOLYGON (((-122.84 49, -120 49, -117.0312...",USA,POINT (-99.31483 37.23674)
21,India,"POLYGON ((97.32711 28.26158, 97.40256 27.88254...",IND,POINT (79.17906 21.87221)
22,China,"MULTIPOLYGON (((109.47521 18.1977, 108.65521 1...",CHN,POINT (98.76958 36.79871)
37,UAE,"POLYGON ((51.57952 24.2455, 51.75744 24.29407,...",ARE,POINT (54.98792 24.28184)
71,Brazil,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",BRA,POINT (-49.71162 -14.07369)
90,South Africa,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",ZAF,POINT (26.14763 -28.40852)
133,Russia,"MULTIPOLYGON (((178.7253 71.0988, 180 71.51571...",RUS,POINT (88.59733 59.40587)
153,UK,"MULTIPOLYGON (((-6.19788 53.86757, -6.95373 54...",GBR,POINT (-1.75331 54.22469)
157,Switzerland,"POLYGON ((9.59423 47.52506, 9.63293 47.3476, 9...",CHE,POINT (8.28652 46.80966)
9,Singapore,"MULTIPOLYGON (((103.56556 1.19556, 103.57364 1...",SGP,POINT (103.86084 1.30335)


In [147]:
data = data.merge(gdf_countries, left_on='Country', right_on='admin', how='left')
data

,Transaction ID,Country,Amount (USD),Transaction Type,Date of Transaction,Person Involved,Industry,Destination Country,Reported by Authority,Source of Money,Money Laundering Risk Score,Shell Companies Involved,Financial Institution,Tax Haven Country,Date,admin,geometry,iso_a3,rep_point
0,TX0000000001,Brazil,3.267530e+06,Offshore Transfer,2013-01-01 00:00:00,Person_1101,Construction,USA,True,Illegal,6,1,Bank_40,Singapore,2013-01-01,Brazil,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",BRA,POINT (-49.71162 -14.07369)
1,TX0000000002,China,4.965767e+06,Stocks Transfer,2013-01-01 01:00:00,Person_7484,Luxury Goods,South Africa,False,Illegal,9,0,Bank_461,Bahamas,2013-01-01,China,"MULTIPOLYGON (((109.47521 18.1977, 108.65521 1...",CHN,POINT (98.76958 36.79871)
2,TX0000000003,UK,9.416750e+04,Stocks Transfer,2013-01-01 02:00:00,Person_3655,Construction,Switzerland,True,Illegal,1,3,Bank_387,Switzerland,2013-01-01,UK,"MULTIPOLYGON (((-6.19788 53.86757, -6.95373 54...",GBR,POINT (-1.75331 54.22469)
3,TX0000000004,UAE,3.864201e+05,Cash Withdrawal,2013-01-01 03:00:00,Person_3226,Oil & Gas,Russia,False,Illegal,7,2,Bank_353,Panama,2013-01-01,UAE,"POLYGON ((51.57952 24.2455, 51.75744 24.29407,...",ARE,POINT (54.98792 24.28184)
4,TX0000000005,South Africa,6.433784e+05,Cryptocurrency,2013-01-01 04:00:00,Person_7975,Real Estate,USA,True,Illegal,1,9,Bank_57,Luxembourg,2013-01-01,South Africa,"POLYGON ((16.34498 -28.57671, 16.82402 -28.082...",ZAF,POINT (26.14763 -28.40852)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,TX0000009996,Singapore,2.209229e+06,Offshore Transfer,2014-02-21 11:00:00,Person_6896,Real Estate,Brazil,False,Illegal,4,4,Bank_258,Luxembourg,2014-02-21,Singapore,"MULTIPOLYGON (((103.56556 1.19556, 103.57364 1...",SGP,POINT (103.86084 1.30335)
9996,TX0000009997,UAE,4.800338e+06,Property Purchase,2014-02-21 12:00:00,Person_6348,Luxury Goods,Russia,False,Legal,10,2,Bank_449,Luxembourg,2014-02-21,UAE,"POLYGON ((51.57952 24.2455, 51.75744 24.29407,...",ARE,POINT (54.98792 24.28184)
9997,TX0000009998,UK,4.891953e+05,Stocks Transfer,2014-02-21 13:00:00,Person_4171,Oil & Gas,Russia,False,Illegal,5,0,Bank_156,Cayman Islands,2014-02-21,UK,"MULTIPOLYGON (((-6.19788 53.86757, -6.95373 54...",GBR,POINT (-1.75331 54.22469)
9998,TX0000009999,Brazil,2.233581e+06,Offshore Transfer,2014-02-21 14:00:00,Person_2799,Real Estate,Russia,True,Illegal,10,5,Bank_12,Singapore,2014-02-21,Brazil,"POLYGON ((-53.37366 -33.76838, -53.65054 -33.2...",BRA,POINT (-49.71162 -14.07369)


In [148]:
iso_a3_dict = data[['Country', 'iso_a3']].drop_duplicates().set_index('Country')['iso_a3'].to_dict()
iso_a3_dict

{'Brazil': 'BRA',
 'China': 'CHN',
 'UK': 'GBR',
 'UAE': 'ARE',
 'South Africa': 'ZAF',
 'Russia': 'RUS',
 'Switzerland': 'CHE',
 'India': 'IND',
 'USA': 'USA',
 'Singapore': 'SGP'}

In [149]:
import plotly.graph_objects as go
import random
from plotly.subplots import make_subplots

#create a dict to map each coutry to a color
country_color = {
    'USA': 'blue',
    'ZAF': 'orange',
    'CHE': 'green',
    'RUS': 'red',
    'BRA': "#7FE956",
    'GBR': 'brown',
    'IND': 'pink',
    'CHN': 'gray',
    'SGP': 'cyan',
    'ARE': 'magenta'
}

colorscale = [
    [0.0, '#FFFFB2'],   # light yellow
    [0.25, '#FECC5C'],  # yellow-orange
    [0.5, '#FD8D3C'],   # orange
    [0.75, '#F03B20'],  # red-orange
    [1.0, '#BD0026']    # dark red
]

def get_color(amount, min_amt, max_amt):
    # Normalize amount to [0, 1] and map to colorscale
    if max_amt == min_amt:
        idx = 0
    else:
        idx = int((amount - min_amt) / (max_amt - min_amt) * (len(colorscale) - 1))
    return colorscale[idx][1]

def make_transaction_map(df, selected_date, origin_options=['origin', 'destiny'], country='ALL'):
    # Filtramos por la fecha seleccionada
    selected_date = pd.to_datetime(selected_date).date()
    transaction = df[df['Date'] == selected_date]

    if country != 'ALL':
        transaction = transaction[(transaction['Country'] == country) | (transaction['Destination Country'] == country)]
        if 'origin' in origin_options and 'destiny' not in origin_options:
            transaction = transaction[transaction['Country'] == country]
        elif 'destiny' in origin_options and 'origin' not in origin_options:
            transaction = transaction[transaction['Destination Country'] == country]
        else:
            transaction = transaction[(transaction['Country'] == country) | (transaction['Destination Country'] == country)]

    # Obtenemos coordenadas (lat/lon) a partir del punto representativo
    transaction_info = transaction[['iso_a3', 'rep_point', 'Destination Country', 'Amount (USD)']].drop_duplicates()

    transaction_info['rep_point_origin'] = transaction_info['rep_point']
    transaction_info['rep_point_destination'] = transaction_info['Destination Country'].map(gdf_countries.set_index('admin')['rep_point'])

    transaction_info['o_lat'] = transaction_info['rep_point_origin'].apply(lambda point: point.y)
    transaction_info['o_lon'] = transaction_info['rep_point_origin'].apply(lambda point: point.x)
    transaction_info['d_lat'] = transaction_info['rep_point_destination'].apply(lambda point: point.y)
    transaction_info['d_lon'] = transaction_info['rep_point_destination'].apply(lambda point: point.x)

    transaction_info['iso_a3_dest'] = transaction_info['Destination Country'].map(iso_a3_dict)

    # obtain origin and destination coordinates
    flows_df = pd.DataFrame(transaction_info)
    flows = flows_df.rename(columns={'Amount (USD)': 'amount', 'iso_a3': 'origin_iso_a3', 'iso_a3_dest': 'dest_iso_a3'})

    show_arrows = True
    if 'origin' in origin_options and 'destiny' in origin_options:
        total_received = flows.groupby('dest_iso_a3')['amount'].sum()
        total_sent = flows.groupby('origin_iso_a3')['amount'].sum() * -1
        total = total_received.add(total_sent, fill_value=0)
    elif 'origin' in origin_options:
        total = flows.groupby('origin_iso_a3')['amount'].sum() * -1
    elif 'destiny' in origin_options:
        total = flows.groupby('dest_iso_a3')['amount'].sum()
    else:
        total = pd.Series(np.zeros(len(gdf_countries)), index=gdf_countries['iso_a3'])
        show_arrows = False
    min_amt, max_amt = total.min(), total.max()
    fig = go.Figure()
    # Color each country based on the total amount received
    amount_colors = {}
    for iso, amt in total.items():
        amount_colors[iso] = get_color(amt, min_amt, max_amt)

    # Add choropleth layer for countries
    fig.add_trace(go.Choropleth(
        locations=list(amount_colors.keys()),
        z=list(total.values),
        text=[admin for admin in gdf_countries.set_index('iso_a3').loc[list(amount_colors.keys()), 'admin']],
        colorscale=[c[1] for c in colorscale],
        autocolorscale=True,
        marker_line_color='white',
        colorbar_title="Flow Amount (USD)",
        geo='geo',
        showscale=True
    ))

    traces = []
    for _, r in flows.iterrows():
        dest_color = get_color(total.get(r['dest_iso_a3'], min_amt), min_amt, max_amt)
        traces.append(go.Scattergeo(
            lon=[r['o_lon'], r['d_lon'] + random.uniform(-1.5, 1.5)],
            lat=[r['o_lat'], r['d_lat'] + random.uniform(-1.5, 1.5)],
            mode='lines+markers',
            line=dict(width=1 + r['amount'] / 1000000, color=country_color.get(r['origin_iso_a3'], 'black')),
            marker=dict(
                size=[0, 8 + r['amount'] / 500000],
                symbol=['circle', 'triangle-up'],
                color=['blue', country_color.get(r['origin_iso_a3'], 'black')],
                line=dict(width=[0, 0], color=['blue', country_color.get(r['origin_iso_a3'], 'black')])
            ),
            opacity=0.7,
            hoverinfo='text',
            text=[
            f"Origen: {r['origin_iso_a3']} ({gdf_countries.set_index('iso_a3').loc[r['origin_iso_a3'], 'admin']})<br>"
            f"Destino: {r['dest_iso_a3']} ({gdf_countries.set_index('iso_a3').loc[r['dest_iso_a3'], 'admin']})<br>"
            f"Flujo: -{r['amount']/1e6:03f} Millions (USD)",
            f"Origen: {r['origin_iso_a3']} ({gdf_countries.set_index('iso_a3').loc[r['origin_iso_a3'], 'admin']})<br>"
            f"Destino: {r['dest_iso_a3']} ({gdf_countries.set_index('iso_a3').loc[r['dest_iso_a3'], 'admin']})<br>"
            f"Flujo: +{r['amount']/1e6:03f} Millions (USD)"
            ],
            showlegend=False
        ))

    if show_arrows:
        for trace in traces:
            fig.add_trace(trace)

    legend_traces = []
    for iso, color in country_color.items():
        admin_name = gdf_countries.set_index('iso_a3').loc[iso, 'admin'] if iso in gdf_countries['iso_a3'].values else iso
        legend_traces.append(go.Scattergeo(
            lon=[None], lat=[None],  # invisible points
            mode='markers',
            marker=dict(size=12, color=color),
            name=admin_name,
            text=f"Send by {admin_name}",
            showlegend=True
        ))

    if show_arrows:
        for trace in legend_traces:
            fig.add_trace(trace)

    fig.update_layout(
        legend=dict(
            orientation="v",
            yanchor="top",
            y=1,
            xanchor="left",
            x=0,
            bgcolor="rgba(255,255,255,0.7)"
        )
    )

    fig.update_geos(
        projection_type='natural earth',
        showcountries=True,
        showcoastlines=True,
        showland=True,
        showocean=True,
        showlakes=True,
        oceancolor="#2391FF",
        lakecolor="#82C0FF",
        landcolor="#C2A580",
        coastlinecolor="#A0A0A0",
        resolution=110
    )
    fig.update_layout(
        margin=dict(l=0, r=0, t=0, b=0),
        hovermode='closest',
        title=f"Transacciones del {selected_date}"
    )
    return fig, flows

def make_transaction_over_time(selected_industries, country_selected, data=data):
    filtered_data = data[data['Industry'].isin(selected_industries) & (data['Country'].isin(country_selected))]
    transactions_over_time = filtered_data.groupby(['Country', 'Date'])['Amount (USD)'].sum().reset_index()

    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"colspan": 2}, None], [{}, {}]],
        subplot_titles=(
            "Transaction Amount Over Time by Country",
            "Total Transactions by Industry and Destination Country"
        ),
        column_widths=[0.7, 0.3],
        row_heights=[1.5, 1.5]
    )

    # Fig 1: Transaction amount over time by country
    for country in transactions_over_time['Country'].unique():
        country_data = transactions_over_time[transactions_over_time['Country'] == country]
        fig.add_trace(go.Scatter(
            x=country_data['Date'],
            y=country_data['Amount (USD)'],
            mode='lines+markers',
            name=country,
            line=dict(color=country_color.get(iso_a3_dict.get(country, ''), 'black')),
        ), row=1, col=1)

    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="Total Amount (USD)", row=1, col=1)
    fig.update_layout(title="Transaction Amount Over Time by Country")
    fig.update_layout(barmode='stack', title="Total Transactions by Industry and Destination Country")

    # Fig 2: Total of transactions by industry. Stack bar plot, each stack is a destination country
    industry_totals = filtered_data.groupby(['Country', 'Destination Country'])['Amount (USD)'].sum().reset_index()
    for des_country in industry_totals['Destination Country'].unique():
        des_country_data = industry_totals[industry_totals['Destination Country'] == des_country]
        fig.add_trace(go.Bar(
            x=des_country_data['Country'],
            y=des_country_data['Amount (USD)'] / 1e6,  # Convert to millions
            name=des_country,
            marker=dict(color=country_color.get(iso_a3_dict.get(des_country, ''), 'black')),
            hovertemplate=f"Destination: {des_country}<br>Total Amount: %{{y:.2f}} (Millions USD)<extra></extra>"
        ), row=2, col=1)
    #set yticks to 0, 1, 2, ..., max y rounded up
    fig.update_yaxes(title_text="Total Amount (Millions USD)", row=2, col=1)
    fig.update_xaxes(title_text="Origin Country", row=2, col=1)
    fig.update_layout(barmode='stack')

    return fig


In [150]:
data['Transaction Type'].unique()

array(['Offshore Transfer', 'Stocks Transfer', 'Cash Withdrawal',
       'Cryptocurrency', 'Property Purchase'], dtype=object)

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt
import io
import base64
from plotly.subplots import make_subplots

app = dash.Dash(__name__)

colorscale = [
    [0.0, "#AEC6CF"],  # pastel blue
    [0.25, "#FFB347"], # pastel orange
    [0.5, "#77DD77"],  # pastel green
    [0.75, "#FFB6C1"], # pastel pink
    [1.0, "#CBAACB"]   # pastel purple
]

color_transaction_type = {
    "Offshore Transfer": "#AEC6CF",  # pastel blue
    "Cash Withdrawal": "#FFB347",      # pastel orange
    "Cryptocurrency": "#77DD77",       # pastel green
    "Stocks Transfer": "#FFB6C1",      # pastel pink
    "Property Purchase": "#CBAACB"     # pastel purple
}

app.layout = html.Div([
    html.Iframe(id='reported-map', style={'width': '100%', 'height': '600px'}),
    #dropdown for selecting the country
    # make a calendar day selector
    dcc.DatePickerSingle(
        id='date-picker',
        date=data['Date'].min(),
        display_format='YYYY-MM-DD',
        min_date_allowed=data['Date'].min(),
        max_date_allowed=data['Date'].max()
    ),
    # replace button with checkboxes for transaction type selection
    dcc.Checklist(
        id='transaction-checklist',
        options=[
            {'label': 'Origin Transactions', 'value': 'origin'},
            {'label': 'Destiny Transactions', 'value': 'destiny'}
        ],
        value=['origin', 'destiny'],
        inline=True
    ),
    dcc.Dropdown(id='country-selector',
        options=[{'label': country, 'value': country} for country in data['Country'].unique()] + [{'label': 'ALL', 'value': 'ALL'}],
        value='ALL',
        clearable=False
    ),
    dcc.Graph(id='transaction-arrow-map', style={'width': '100%', 'height': '400px'}),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in data['Country'].unique()],
        value='USA',
        clearable=False
    ),
    html.Button('Normalize', id='normalize-button', n_clicks=0, style={'margin': '10px'}),
    dcc.Graph(id='industry-bar-chart'),
    #dropdown by industry
    dcc.Dropdown(id='industry-dropdown',
        options=[{'label': industry, 'value': industry} for industry in data['Industry'].unique()],
        value=data['Industry'].unique().tolist(),
        multi=True,
        clearable=False
    ),
    dcc.Dropdown(id='country-dropdown-multi', 
        options=[{'label': country, 'value': country} for country in data['Country'].unique()],
        value=data['Country'].unique().tolist(),
        multi=True,
        clearable=False
    ),
    dcc.Graph(id='transactions-over-time', style={'width': '100%', 'height': '600px'})
])

@app.callback(
    Output('reported-map', 'srcDoc'), 
    Input('reported-map', 'id')
)
def update_map(id, dataset=data, geo_data=gdf_countries):
    clean_data = dataset[['Country', 'Reported by Authority', 'Source of Money', 'Amount (USD)', 'Transaction Type']]
    clean_data_illegal = clean_data[clean_data['Source of Money'] == 'Illegal']
    data_country = clean_data_illegal.groupby('Country')
    map_illegal_data = {}
    map_transactions_data = {}
    for country, group in data_country:
        illegal_count = group['Reported by Authority'].sum()
        illegal_total = len(group)
        map_illegal_data[country] = illegal_count / illegal_total if illegal_total > 0 else 0

        transaction_counts = group['Transaction Type'].value_counts().to_dict()
        map_transactions_data[country] = transaction_counts

    map = folium.Map(location=[20, 0], zoom_start=2)
    folium.Choropleth(
        geo_data=geo_data[['admin', 'geometry']],
        name='choropleth',
        data=clean_data_illegal.groupby('Country')['Amount (USD)'].sum() / 1e6,
        columns=['admin', 'Illegal Ratio'],
        key_on='feature.properties.admin',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Total Illegal Amount (Millions USD)',
    ).add_to(map)

    fg_markers = folium.FeatureGroup(name="centroid_markers_illegal")
    for country, ratio in map_illegal_data.items():
        geometry = geo_data[geo_data['admin'] == country].geometry
        if geometry.empty:
            continue
        centroid = geometry.values[0].centroid

        fig, ax = plt.subplots(figsize=(2, 2), dpi=150)
        ax.pie([ratio, 1 - ratio], colors=[colorscale[int(ratio * (len(colorscale) - 1))][1], "#FF6A6A"], startangle=90)
        ax.axis('equal')
        ax.set_title(f"{country}\nIllegal Ratio: {ratio:.2%}", fontsize=8)

        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight')  # guarda como PNG in-memory
        plt.close(fig)
        img_base64 = base64.b64encode(buf.getvalue()).decode("utf-8")

        html = f"""
        <div style="text-align:center;">
            <img src="data:image/png;base64,{img_base64}" style="max-width:100%; height:auto;" />
        </div>
        """
        iframe_popup = folium.IFrame(html=html, width=150, height=170)
        popup = folium.Popup(iframe_popup, max_width=300)

        folium.Marker(
            location=[centroid.y, centroid.x],
            radius=5 + ratio * 20,
            popup=popup,
            color='blue',
            
        ).add_to(fg_markers)

    fg_markers.add_to(map)

    fg_markers = folium.FeatureGroup(name="centroid_markers_transactions")
    for country, ratio in map_illegal_data.items():
        geometry = geo_data[geo_data['admin'] == country].geometry
        if geometry.empty:
            continue
        centroid = geometry.values[0].centroid

        fig, ax = plt.subplots(figsize=(2, 2), dpi=150)
        ax.bar(map_transactions_data[country].keys(), map_transactions_data[country].values(), color=[color_transaction_type.get(txn_type, "#FFFFFF") for txn_type in map_transactions_data[country].keys()])
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.set_xticklabels(map_transactions_data[country].keys(), rotation=45, fontsize=8)
        ax.set_title(f"Transactions in {country}", fontsize=8)

        buf = io.BytesIO()
        fig.savefig(buf, format='png', bbox_inches='tight')  # guarda como PNG in-memory
        plt.close(fig)
        img_base64 = base64.b64encode(buf.getvalue()).decode("utf-8")

        html = f"""
        <div style="text-align:center;">
            <img src="data:image/png;base64,{img_base64}" style="max-width:100%; height:auto;" />
        </div>
        """
        iframe_popup = folium.IFrame(html=html, width=150, height=170)
        popup = folium.Popup(iframe_popup, max_width=300)

        folium.Marker(
            location=[centroid.y, centroid.x],
            radius=5 + ratio * 20,
            popup=popup,
            color='blue',
            
        ).add_to(fg_markers)

    fg_markers.add_to(map)
    
    folium.LayerControl().add_to(map)

    return map._repr_html_()

@app.callback(
    Output('industry-bar-chart', 'figure'),
    Input('country-dropdown', 'value'),
    Input('normalize-button', 'n_clicks')
)
def update_bar_chart(selected_country, normalize_clicks=0, dataset=data):
    filtered_data = dataset[dataset['Country'] == selected_country]
    filtered_data['Amount (USD)'] = filtered_data['Amount (USD)'] / 1e6
    illegal_data = filtered_data[filtered_data['Source of Money'] == 'Illegal']
    legal_data = filtered_data[filtered_data['Source of Money'] == 'Legal']

    # Agrupa por industria
    industry_group_illegal = illegal_data.groupby('Industry')
    industry_group_legal = legal_data.groupby('Industry')
    
    #illegal data
    illegal_counts = industry_group_illegal.size().reset_index(name='Illegal Transaction Count')
    illegal_amounts = industry_group_illegal['Amount (USD)'].sum().reset_index(name='Illegal Amount (USD)')
    illegal_amounts['Illegal Amount (Millions USD)'] = illegal_amounts['Illegal Amount (USD)']

    # legal_data
    legal_counts = industry_group_legal.size().reset_index(name='Legal Transaction Count')
    legal_amounts = industry_group_legal['Amount (USD)'].sum().reset_index(name='Legal Amount (USD)')
    legal_amounts['Legal Amount (Millions USD)'] = legal_amounts['Legal Amount (USD)']

    # Subplots: bar chart (count) + bar chart (amount)
    fig = make_subplots(
        rows=1, cols=2, subplot_titles=['Transaction Count', 'Amount (Millions USD)'],
        shared_yaxes=False
    )
    suffix = ""
    if normalize_clicks % 2 == 1:
        suffix = " (Normalized)"

        # Normalize counts
        total_counts = illegal_counts['Illegal Transaction Count'] + legal_counts['Legal Transaction Count']
        illegal_counts['Illegal Transaction Count'] = illegal_counts['Illegal Transaction Count'] / total_counts
        legal_counts['Legal Transaction Count'] = legal_counts['Legal Transaction Count'] / total_counts

        # Normalize amounts
        total_amounts = illegal_amounts['Illegal Amount (USD)'] + legal_amounts['Legal Amount (USD)']
        # Avoid division by zero and align indices
        total_amounts = total_amounts.replace(0, np.nan)
        illegal_amounts['Illegal Amount (Millions USD)'] = illegal_amounts['Illegal Amount (USD)'] / total_amounts
        legal_amounts['Legal Amount (Millions USD)'] = legal_amounts['Legal Amount (USD)'] / total_amounts

    # Add traces for count subplot
    fig.add_trace(go.Bar(
        x=illegal_counts['Industry'],
        y=illegal_counts['Illegal Transaction Count'],
        name=f'Illegal{suffix}',
        marker_color="#FF4747",
        legendgroup='group1',
        showlegend=True,
        texttemplate='%{y:.2%}' if normalize_clicks % 2 == 1 else '%{y}',
        textposition='auto'
    ), row=1, col=1)

    fig.add_trace(go.Bar(
        x=legal_counts['Industry'],
        y=legal_counts['Legal Transaction Count'],
        name=f'Legal{suffix}',
        marker_color='#77DD77',
        legendgroup='group2',
        showlegend=True,
        texttemplate='%{y:.2%}' if normalize_clicks % 2 == 1 else '%{y}',
        textposition='auto'
    ), row=1, col=1)

    # Add traces for amount subplot, but don't show legend again
    fig.add_trace(go.Bar(
        x=illegal_amounts['Industry'],
        y=illegal_amounts['Illegal Amount (Millions USD)'],
        name=f'Illegal{suffix}',
        marker_color="#FF4747",
        legendgroup='group1',
        showlegend=False,
        texttemplate='%{y:.2%}' if normalize_clicks % 2 == 1 else '%{y:.2f}',
        textposition='auto'
    ), row=1, col=2)

    fig.add_trace(go.Bar(
        x=legal_amounts['Industry'],
        y=legal_amounts['Legal Amount (Millions USD)'],
        name=f'Legal{suffix}',
        marker_color='#77DD77',
        legendgroup='group2',
        showlegend=False,
        texttemplate='%{y:.2%}' if normalize_clicks % 2 == 1 else '%{y:.2f}',
        textposition='auto'
    ), row=1, col=2)

    fig.update_layout(
        title='Transaction Overview by Industry',
        template='simple_white',
        legend_title_text='Source of Money',
        barmode='stack'
    )

    fig.update_xaxes(title_text='Industry', row=1, col=1)
    fig.update_yaxes(title_text='Count', row=1, col=1)
    fig.update_xaxes(title_text='Industry', row=1, col=2)
    fig.update_yaxes(title_text='Amount (Millions USD)', row=1, col=2)

    return fig


@app.callback(
    Output('transaction-arrow-map', 'figure'),
    Input('date-picker', 'date'),
    Input('transaction-checklist', 'value'),
    Input('country-selector', 'value')
)
def update_arrow_map(selected_date, origin_options, selected_country):
    # show_arrows = toggle_arrows % 2 == 1  # Toggle arrows on/off based on odd/even clicks
    selected_date = pd.to_datetime(selected_date).date()
    if selected_date is None or selected_date < data['Date'].min() or selected_date > data['Date'].max():
        selected_date = data['Date'].min()
    fig, flows = make_transaction_map(data, selected_date, origin_options=origin_options, country=selected_country)
    return fig

@app.callback(
    Output('transactions-over-time', 'figure'),
    Input('industry-dropdown', 'value'),
    Input('country-dropdown-multi', 'value')
)
def update_transactions_over_time(selected_industries, selected_country):
    fig = make_transaction_over_time(selected_industries, selected_country)
    return fig

app.run(debug=True, port=8050)
    

C:\Users\mario\AppData\Local\Temp\ipykernel_21140\398724313.py:195: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\mario\AppData\Local\Temp\ipykernel_21140\398724313.py:158: UserWarning:

set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.

C:\Users\mario\AppData\Local\Temp\ipykernel_21140\398724313.py:158: UserWarning:

set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.

C:\Users\mario\AppData\Local\Temp\ipykernel_21140\398724313.py:158: UserWarning:

set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.

C:\Users\mario\AppData\Local\Temp\ipykernel_2114